# Introduction to Genki Signals

### Signal Sources and Samplers

In [1]:
!pwd

/Users/egill/dev/genki/genki-signals/examples


In [2]:
import os

# this isn't required with library pip installed
os.chdir('../')

The first building block of Genki Signals is the `SignalSource`

SignalSource is a callable that returns the current value of that Signal. It can be a function or a class with a `__call__` method. One example is the `MouseSignalSource` that gives the current position of the pointer:

In [3]:
from genki_signals.signal_sources import MouseSignalSource

source = MouseSignalSource()
source()

array([511.67578125, 406.46875   ])

Obviously we want to gather multiple samples of this signal. For that we need a `Sampler`.

A Sampler simply samples one or many SignalSources at a given _sample rate_:

In [4]:
from genki_signals.signal_sources import Sampler

# sources is a dictionary mapping names to signal esources
sampler = Sampler(sources = {"mouse": source}, sample_rate=100)

Now the sampler samples the mouse position at a rate of 100 samples / second (hz) (This happens in a separate thread so the cell still returns and the main thread is unblocked.)

Signal Sources and Samplers are the way to get raw data into Genki Signals, usually from some external source, like a web API or an external device. Sometimes these will be combined into a single class, e.g. a microphone that samples audio data at a specific sample rate which we have no control over, but more on that later.

### Signal Functions and Systems

We might want to do some processing on the samples from our sampler. 

This is where SignalFunctions come in. SignalFunctions are functions that take in one or more signals and return another signal.

There is a collection of SignalFunctions available in `genki_signals.signal_functions` and they all have a similar structure, to create one you need to specify some _input names_ and also a _name_ for the output signal: 

In [5]:
import genki_signals.signal_functions as sf

# diff differentiates the "mouse" signals with regard to the "timestamp" and returns the signal "mouse_vel"
# the "mouse" signal is the one we created earlier and "timestamp" is created automatically by the sampler

diff = sf.Differentiate(input_a="mouse", input_b="timestamp", name="mouse_vel")

We need to introduce one more concept to connect all of this together: the SignalSystem.

SignalSystem takes in a Sampler/SignalSource and a list of SignalFunctions

In [6]:
from genki_signals.signal_system import SignalSystem

system = SignalSystem(sampler, [diff])

system.start()

The `signal_functions` module contains a library of functions to do signal arithmetic, digital signal processing, e.g. filtering, geometric calculations (useful for IMU sensors), create basic waveforms, run real time inference with machine learningh models, and more. We will dive into these in depth later.

### Visualization

This is the minimal setup we need. With a system, we can add signal functions to do all kinds of processing, and we can do data recording to start building a dataset. However, probably the most useful part of Genki Signals is the real-time visualization. For that we need a `Frontend`: 

In [7]:
from genki_signals.signal_frontends import WidgetFrontend, Line

l_position = Line("timestamp", "mouse")
l_velocity = Line("timestamp", "mouse_vel")

dashboard = WidgetFrontend(system, widgets=[l_position, l_velocity])
dashboard

registering data feed


Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widgets
Updating widge

A `WidgetFrontend` is a front end specifically designed for jupyter notebooks. In the above cell we create two `Line` widgets and combine them into a `dashboard` object that is then rendered in a notebook. A Genki Signals frontend can also be a separate web server or any sort of GUI to visualize and interact with the system. 

# Recording data

Our `dashboard` object displays the data in real time. The last basic feature we will introduce is recording. To start recording data, we can simply call `system.start_recording()` and give it a filename:

In [27]:
system.start_recording('session_1')

The name `session_1` will be the name of a folder. We'll see its contents in a bit.

We are now recording data, try moving the mouse around a bit and run the next cell to stop recording (and stop everything):

In [9]:
system.stop_recording()
system.stop()

Updating widgets
Updating widgets


The system created the folder `session_1`, let's see what it contains:

In [29]:
!ls session_1/

metadata.json   raw_data.pickle


`raw_data.pickle` contains the recorded data, whereas `metadata.json` contains various information about this recording session. Instead of reading these files directly we can load them in a `Session`:

In [30]:
from genki_signals.session import Session

session = Session.from_filename('session_1')
session.data['mouse'].shape

(2, 1652)

So we have a dataset of mouse positions! 

Note that we don't actually store the differentiated signal `mouse_vel`, we only store the raw data which is treated as a source of truth. The `SignalFunction`s are serialized and stored in `metadata.json`. All `SignalFunction`s are deterministic so they can be recomputed at will. This means that the signal functions (and their parameters, if any) can be used as hyperparameters in ML training. For example, if you want to use a low-pass filter on some signal, the exact cutoff frequency of the filter can be treated as a hyperparameter.

The `metadata.json` can also contain arbitrary information about the particular session. This can also be useful in a machine learning setting if you wish to, for example, make sure the data is split into train and test based on _individuals_, so that no individual who appears in the test set has any data in the training set. 

To recap, we have introduced a lot of concepts:

* A `SignalSource` is some way to get an external signal
* A `Sampler` samples values from a `SignalSource` at a given rate
* A `SignalFunction` is some function to process signals
* A `SignalSystem` ties all of the above together and records data
* A `Session` is some data that was recorded in a single recording session
* A `Frontend` is a way to visualize what is going on in a system and to interact with it

We have only scratched the surface of these components. This should be enough to get started but in the following notebooks we will cover each of these in more depth.